## Removal of pig and human sequences:
Pig and human metagenomes contain host contamination (most likely). 
We need to at least make a not of what % of each dataset is contamination, and ideally remove contamination from each before using it for other downstream analysis. 

Have tried sig subtract , but in order to use that I needed to flatten the signatures, which I think messed with downstream analysis.

I would like to write a conscise snakefile that for each signature removes these kmers and outputs a gather csv with f_orig_query (perc of metaG that matches contaminant genome). 

Snakefile needs to be able to take in lists of SRA numbers, and based off that put output in the correct folder (pig or human, maybe also distinguish between yes/no mags built). And for input files there are different paths as well. 

Steps:
1. fastmultigather metaG x  pig genome
2. fastmultigather metaG x  human genome
3. concat these output files
4. gather x metagenome with output from 2 as a picklist
5. use --output-unassigned and save as zip. 
6. Save a csv output as well of gather matches. 

In [ ]:
# how to run snakefile:
# Run snakefile that concatenates fmg output, then uses that as a picklist
# srun snakemake
srun --account=ctbrowngrp -p med2 -J fmg -t 5:00:00 -c 50 --mem=50gb --pty bash
mamba activate branchwater

snakemake -s remove_contam.smk --use-conda --resources mem_mb=50000 --rerun-triggers mtime \
-c 40 --rerun-incomplete -k -n

In [ ]:
# make signature list:
# sigs in: /group/ctbrowngrp/irber/data/wort-data/wort-sra/sigs/{sig}.sig
awk '{print "/group/ctbrowngrp/irber/data/wort-data/wort-sra/sigs/" $0}' ../../config/human_sra.txt > human_sra.txt
awk '{print $0 ".sig"}' human_sra.txt > human_sra.fp.txt

# for pigs from which no MAGs were built
awk '{print "/group/ctbrowngrp/irber/data/wort-data/wort-sra/sigs/" $0}' ../../config/pig_sra_noatlas.txt > pig_sra_noatlas.txt
awk '{print $0 ".sig"}' pig_sra_noatlas.txt > pig_sra_noatlas.fp.txt

# Make one file since we dont care atm if its human or pig metag
cat human_sra.fp.txt pig_sra_noatlas.fp.txt > filepath_wortsigs.txt

# same thing for pig gut from which we have MAGs
awk '{print "/group/ctbrowngrp2/scratch/annie/2024-pigparadigm/results/sourmash/sketches/metag_hostrm/" $0}' ../../config/pig_sra.txt > pig_sra.txt
awk '{print $0 ".sig.gz"}' pig_sra.txt > pig_sra.fp.txt


In [ ]:
# fastmultigather, just srun it
srun --account=ctbrowngrp -p med2 -J fmg -t 10:00:00 -c 50 --mem=50gb --pty bash
mamba activate branchwater-skipmer

# fmg to the pig genome
sourmash scripts fastmultigather \
../all_filepath.txt /group/ctbrowngrp2/scratch/annie/2024-pigparadigm/resources/sus_scrofa/susScr11.sig.gz \
-k 21 -s 1000 -c 50 -t 0

# and to the pig genome
mamba activate branchwater
sourmash scripts fastmultigather \
../all_filepath.txt /group/ctbrowngrp/sourmash-db/hg38/hg38-entire.sig.zip \
-k 21 -s 1000 -c 50 -t 0

In [ ]:
# human db =  /group/ctbrowngrp/sourmash-db/hg38/hg38-entire.sig.zip
# pigdb = '/group/ctbrowngrp2/scratch/annie/2024-pigparadigm/resources/sus_scrofa/susScr11.sig.gz'
